In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
from copulas.multivariate import GaussianMultivariate
from copulas.bivariate import Clayton, Frank, Gumbel
from copulas.bivariate.independence import Independence
from scipy.stats import pearson3, laplace_asymmetric, loglaplace
from scipy.interpolate import griddata
import os

#Define Path where all inputs are located
path =
os.chdir(path)
MPtoT = pd.read_csv('MPtoT.csv')
MTtoR = pd.read_csv('MTtoR.csv')
MRtoP = pd.read_csv('MRtoP.csv')
MPtoR = pd.read_csv('MPtoR.csv')
MRtoT = pd.read_csv('MRtoT.csv')
MTtoP = pd.read_csv('MTtoP.csv')
MTtoR1 = pd.read_csv('MTtoR1.csv')
MTtoR2 = pd.read_csv('MTtoR2.csv')
MTtoR3 = pd.read_csv('MTtoR3.csv')

In [ ]:
#Uniform Data Transformation
def transform_to_uniform(df, distributions):
    uniform_data = pd.DataFrame()
    for col, dist in distributions.items():
        uniform_data[col] = dist.cdf(df[col])
    return uniform_data

def fit_copula_and_plot(df, title, variable_names, Original_data):
    data_array = df.values
    #Define copula types to test 
    copulas = [GaussianMultivariate, Clayton, Frank, Gumbel]

    best_copula = None
    max_likelihood = -np.inf

    for Copula in copulas:
        copula = Copula()
        copula.fit(data_array)
        likelihood = np.sum(np.log(copula.pdf(data_array)))
        # Print Log-Likelihood for specific copulas
        print(f"{Copula.__name__} Log-Likelihood: {likelihood}")
        
        # Print theta parameter for specific copulas
        if isinstance(copula, Clayton):
            print(f"{Copula.__name__} Theta parameter: {copula.theta}")
        elif isinstance(copula, Frank):
            print(f"{Copula.__name__} Theta parameter: {copula.theta}")
        elif isinstance(copula, Gumbel):
            print(f"{Copula.__name__} Theta parameter: {copula.theta}")
            
        if likelihood > max_likelihood:
            max_likelihood = likelihood
            best_copula = Copula

    if best_copula is not None:
        print(f"Best Copula for {title}: {best_copula.__name__}")
        
        fig, ax = plt.subplots()
        #Can change third value for x and y to adjust resolution and run time
        x = np.linspace(0, 1, 1000)
        y = np.linspace(0, 1, 1000)
        X, Y = np.meshgrid(x, y)
        XY = np.dstack((X, Y)).reshape(-1, 2)
        
        best_copula_instance = best_copula()
        best_copula_instance.fit(data_array)
        #Empirical CDF
        emp_cdf = []
        for i in range(0, len(df)):
            s = df.iloc[:, 0][i]
            t = df.iloc[:, 1][i]
            bunch = df.loc[(df.iloc[:, 0] <= s) & (df.iloc[:, 1] <= t)]
            emp_cdf.append(len(bunch) / len(df))
        


        s = df.iloc[:, 0].values
        t = df.iloc[:, 1].values
        z = emp_cdf
        #Define interpolation type under method
        zi = griddata((s, t), z, (x[None, :], y[:, None]), method='cubic')
        #Can change third value for x1 and y1 to adjust resolution and run time. Should be set same as x and y above.
        x1 = np.linspace(0, 1, 1000)
        y1 = np.linspace(0, 1, 1000)
        X1, Y1 = np.meshgrid(x1, y1)
        X1Y1 = np.dstack((X1, Y1)).reshape(-1, 2)
        
        copula_cdf = best_copula_instance.cumulative_distribution(X1Y1).reshape(X1.shape)

        #Define Contour Levels    
        levels = (0.1, 0.5, 0.9)

        CS = ax.contour(X1, Y1, copula_cdf, levels=levels, colors='k')

        CSi = ax.contour(X1, Y1, zi, levels=levels, linewidths=1, colors='b')

        ax.set_title(f"{title} - {best_copula.__name__}")

        
        CS_levels = CS.collections  # Contour levels for CS
        CSi_levels = CSi.collections  # Contour levels for CSi
        # Extract coordinates of the contour lines for CS
        CS_coords = [contour.get_paths()[0].vertices if contour.get_paths() else [] for contour in CS_levels]
        # Extract coordinates of the contour lines for CSi
        CSi_coords = [contour.get_paths()[0].vertices if contour.get_paths() else [] for contour in CSi_levels]
        #Back-Transform CS_coords
        back_transformed_CS_coords = []
        for coords in CS_coords:
            back_transformed_coords = []
            for i in range(len(coords)):
                u1, u2 = coords[i]
                x = dists[f"{df.columns[0]}"].ppf(u1)  # Inverse CDF for Variable 1
                y = dists[f"{df.columns[1]}"].ppf(u2)  # Inverse CDF for Variable 2
                back_transformed_coords.append([x, y])
            back_transformed_CS_coords.append(np.array(back_transformed_coords))

        # Back-transform CSi_coords
        back_transformed_CSi_coords = []
        for coords in CSi_coords:
            back_transformed_coords = []
            for i in range(len(coords)):
                u1, u2 = coords[i]
                x = dists[f"{df.columns[0]}"].ppf(u1)  # Inverse CDF for Variable 1
                y = dists[f"{df.columns[1]}"].ppf(u2)  # Inverse CDF for Variable 2
                back_transformed_coords.append([x, y])
            back_transformed_CSi_coords.append(np.array(back_transformed_coords))

        ax.clear()
        empirical_legend_added = False
        theoretical_legend_added = False
        # Plot the back-transformed CS_coords and CSi_coords
        for coords in back_transformed_CS_coords:
            ax.plot(coords[:, 0], coords[:, 1], color='k', linewidth=1)
            if not theoretical_legend_added:
                ax.plot([], [], color='k', linewidth=1, label='Theoretical')  # Add an empty line for the theoretical label
                theoretical_legend_added = True  

        for coords in back_transformed_CSi_coords:
            if coords.shape[0] > 0:  # Check if the array is not empty
                ax.plot(coords[:, 0], coords[:, 1], color='b', linewidth=1) 
                if not empirical_legend_added:
                    ax.plot([], [], color='b', linewidth=1, label='Empirical')  # Add an empty line for the empirical label
                    empirical_legend_added = True
            
           
        # Scatter plot of the original data
        ax.scatter(Original_data.iloc[:, 0], Original_data.iloc[:, 1], marker='o', color='black', alpha=0.5)
        ax.set_xlabel(df.columns[0])
        ax.set_ylabel(df.columns[1])
        ax.set_title(f"Southeast Texas {title} - {best_copula.__name__}")
        # Set axis limits
        if title == 'Max Precipitation to River Discharge':  
            ax.set_ylim(-10, 410)
            ax.set_xlim(20, 360)
        if title == 'Max Still Water to Precipitation':  
            ax.set_xlim(0.6, 1.8)
            ax.set_ylim(-10, 360)
        if title == 'Max Still Water to River Discharge':
            ax.set_xlim(0.6, 1.8)
            ax.set_ylim(-10, 420)
        if title == 'Max River Discharge to Precipitation': 
            ax.set_xlim(-10, 460)
            ax.set_ylim(-10, 290)
        if title == 'Max River Discharge to Still Water':
            ax.set_xlim(20, 420)
            ax.set_ylim(0.2, 1.9)


        plt.tight_layout()
        plt.savefig(f"{title}_copulaGB.png", dpi=1200)
        plt.show()
        # Compute correlation measures
        tau, p_tau = stats.kendalltau(Original_data[Original_data.columns[0]], Original_data[Original_data.columns[1]])
        print(f"Kendall Tau: {tau:.4f} (p-value: {p_tau:.4f})")
        print('------------')
    else:
        print(f"No Copula found for {title}")

#Define Dataframes
dataframes = [MPtoT, MPtoR, MTtoP, MTtoR, MRtoP, MRtoT]
titles = ['Max Precipitation to Still Water', 'Max Precipitation to River Discharge', 'Max Still Water to Precipitation', 
          'Max Still Water to River Discharge', 'Max River Discharge to Precipitation', 'Max River Discharge to Still Water']

# Fit the distributions to the data for each DataFrame
precip_params_MPtoT = pearson3.fit(MPtoT['Precipitation (mm)'])
stillwater_params_MPtoT = laplace_asymmetric.fit(MPtoT['Obs (m above NAVD88)'])

precip_params_MPtoR = pearson3.fit(MPtoR['Precipitation (mm)'])
discharge_params_MPtoR = loglaplace.fit(MPtoR['Flow (m^3/s)'])

stillwater_params_MTtoP = laplace_asymmetric.fit(MTtoP['Obs (m above NAVD88)'])
precip_params_MTtoP = pearson3.fit(MTtoP['Precipitation (mm)'])

stillwater_params_MTtoR = laplace_asymmetric.fit(MTtoR['Obs (m above NAVD88)'])
discharge_params_MTtoR = loglaplace.fit(MTtoR['Flow (m^3/s)'])

discharge_params_MRtoP = loglaplace.fit(MRtoP['Flow (m^3/s)'])
precip_params_MRtoP = pearson3.fit(MRtoP['Precipitation (mm)'])

discharge_params_MRtoT = loglaplace.fit(MRtoT['Flow (m^3/s)'])
stillwater_params_MRtoT = laplace_asymmetric.fit(MRtoT['Obs (m above NAVD88)'])

# Create the fitted distributions
distributions = [
    {'Precipitation (mm)': pearson3(*precip_params_MPtoT), 'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MPtoT)},
    {'Precipitation (mm)': pearson3(*precip_params_MPtoR), 'Flow (m^3/s)': loglaplace(*discharge_params_MPtoR)},
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoP), 'Precipitation (mm)': pearson3(*precip_params_MTtoP)},
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoR), 'Flow (m^3/s)': loglaplace(*discharge_params_MTtoR)},
    {'Flow (m^3/s)': loglaplace(*discharge_params_MRtoP), 'Precipitation (mm)': pearson3(*precip_params_MRtoP)},
    {'Flow (m^3/s)': loglaplace(*discharge_params_MRtoT), 'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MRtoT)},
]

variable_names = ['Precipitation (mm)', 'Obs (m above NAVD88)', 'Flow (m^3/s)']

for df, title, dists in zip(dataframes, titles, distributions):
    uniform_data = transform_to_uniform(df, dists)
    fit_copula_and_plot(uniform_data, title, variable_names, df)
    

In [ ]:
#Very Similar but for SLR
def transform_to_uniform(df, distributions):
    uniform_data = pd.DataFrame()
    for col, dist in distributions.items():
        uniform_data[col] = dist.cdf(df[col])
    return uniform_data

def fit_copula_and_plot(df, uniform_data, title, color, alpha, ax, dists):
    data_array = uniform_data.values
    copulas = [GaussianMultivariate, Clayton, Frank, Gumbel]

    best_copula = None
    max_likelihood = -np.inf

    for Copula in copulas:
        copula = Copula()
        copula.fit(data_array)
        likelihood = np.sum(np.log(copula.pdf(data_array)))

        if likelihood > max_likelihood:
            max_likelihood = likelihood
            best_copula = Copula

    if best_copula is not None:
        print(f"Best Copula for {title}: {best_copula.__name__}")

        best_copula_instance = best_copula()
        best_copula_instance.fit(data_array)
        #can adjust thrid value of x1 and y1 to adjust resolution and run time
        x1 = np.linspace(0, 1, 10000)
        y1 = np.linspace(0, 1, 10000)
        X1, Y1 = np.meshgrid(x1, y1)
        X1Y1 = np.dstack((X1, Y1)).reshape(-1, 2)

        copula_cdf = best_copula_instance.cumulative_distribution(X1Y1).reshape(X1.shape)
        
        levels = (0.1, 0.5, 0.9, 0.99)
        # Plot contours based on original data with the specified color
        CS = ax.contour(X1, Y1, copula_cdf, levels=levels, colors=color, alpha=alpha)

        # Extract coordinates of the contour lines for CS
        CS_coords = [contour.get_paths()[0].vertices if contour.get_paths() else [] for contour in CS.collections]

        # Back-transform CS_coords
        for coords in CS_coords:
            back_transformed_coords = []
            for i in range(len(coords)):
                u1, u2 = coords[i]
                x = dists[f"{uniform_data.columns[0]}"].ppf(u1)
                y = dists[f"{uniform_data.columns[1]}"].ppf(u2)  
                back_transformed_coords.append([x, y])
            back_transformed_CS_coords.append(np.array(back_transformed_coords))
            
        
            
        # Plot the back-transformed CS_coords
        for coords in back_transformed_CS_coords:
                ax.plot(coords[:, 0], coords[:, 1], linewidth=1, color=color, alpha=alpha)
                
        # Remove the contour plot (CS) after using it
        for contour in CS.collections:
            contour.remove()
    else:
        print(f"No Copula found for {title}")
        
# Create a single figure and axis
fig, ax = plt.subplots()
colors = ['red', 'blue', 'blue', 'purple']  # Add colors for each iteration
alpha = (1, 0, 1, 0) #2nd and 4th set to transparent since CI is shaded

dataframes = [MTtoR, MTtoR1, MTtoR2, MTtoR3]
titles = ['MTtoR', 'MTtoR1', 'MTtoR2', 'MTtoR3']

# Fit the distributions to the data for each DataFrame
stillwater_params_MTtoR = laplace_asymmetric.fit(MTtoR['Obs (m above NAVD88)'])
discharge_params_MTtoR = loglaplace.fit(MTtoR['Flow (m^3/s)'])

stillwater_params_MTtoR1 = laplace_asymmetric.fit(MTtoR1['Obs (m above NAVD88)'])
discharge_params_MTtoR1 = loglaplace.fit(MTtoR1['Flow (m^3/s)'])

stillwater_params_MTtoR2 = laplace_asymmetric.fit(MTtoR2['Obs (m above NAVD88)'])
discharge_params_MTtoR2 = loglaplace.fit(MTtoR2['Flow (m^3/s)'])

stillwater_params_MTtoR3 = laplace_asymmetric.fit(MTtoR3['Obs (m above NAVD88)'])
discharge_params_MTtoR3 = loglaplace.fit(MTtoR3['Flow (m^3/s)'])



# Create the fitted distributions
distributions = [
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoR), 'Flow (m^3/s)': loglaplace(*discharge_params_MTtoR)},
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoR1), 'Flow (m^3/s)': loglaplace(*discharge_params_MTtoR1)},
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoR2), 'Flow (m^3/s)': loglaplace(*discharge_params_MTtoR2)},
    {'Obs (m above NAVD88)': laplace_asymmetric(*stillwater_params_MTtoR3), 'Flow (m^3/s)': loglaplace(*discharge_params_MTtoR3)}
]

back_transformed_lines = []  # Store back-transformed lines for each iteration

for df, title, dists, color, alpha_value in zip(dataframes, titles, distributions, colors, alpha):
    uniform_data = transform_to_uniform(df, dists)
    back_transformed_CS_coords = []  # Clear the list for each iteration
    fit_copula_and_plot(df, uniform_data, title, color, alpha_value, ax, dists)
    
    # Store the back-transformed coordinates for this iteration
    back_transformed_lines.append(back_transformed_CS_coords)
    
desired_iteration = 0  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 0  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line = back_transformed_lines[desired_iteration]
ax.plot(desired_line[desired_level_index][:, 0], desired_line[desired_level_index][:, 1], linewidth=1, color='red', label='Current')

desired_iteration = 2  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 0  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line = back_transformed_lines[desired_iteration]
ax.plot(desired_line[desired_level_index][:, 0], desired_line[desired_level_index][:, 1], linewidth=1, color='blue', label='2060 SLR Intermediate Confidence Interval')


# Now you can reference and plot a specific line at a specific level
desired_iteration = 1  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 0  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line1 = back_transformed_lines[desired_iteration]

desired_iteration = 3
desired_level_index = 0
desired_line2 = back_transformed_lines[desired_iteration]

ax.fill_betweenx(desired_line1[desired_level_index][:, 1], desired_line1[desired_level_index][:, 0],
                desired_line2[desired_level_index][:, 0], color='blue', alpha=0.2)
desired_iteration = 1  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 1  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line3 = back_transformed_lines[desired_iteration]

desired_iteration = 3
desired_level_index = 1
desired_line4 = back_transformed_lines[desired_iteration]

ax.fill_betweenx(desired_line3[desired_level_index][:, 1], desired_line3[desired_level_index][:, 0],
                desired_line4[desired_level_index][:, 0], color='blue', alpha=0.2)

desired_iteration = 1  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 2  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line5 = back_transformed_lines[desired_iteration]

desired_iteration = 3
desired_level_index = 2
desired_line6 = back_transformed_lines[desired_iteration]

ax.fill_betweenx(desired_line5[desired_level_index][:, 1], desired_line5[desired_level_index][:, 0],
                desired_line6[desired_level_index][:, 0], color='blue', alpha=0.2)

desired_iteration = 1  # Index of the desired iteration (0 for MPtoT, 1 for MPtoT1, etc.)
desired_level_index = 3  # Index of the desired level (0 for 0.1, 1 for 0.5, 2 for 0.9, 3 for 0.99)

# Plot the desired line from the specified iteration at the desired level
desired_line7 = back_transformed_lines[desired_iteration]

desired_iteration = 3
desired_level_index = 3
desired_line8 = back_transformed_lines[desired_iteration]

ax.fill_between(desired_line7[desired_level_index][:, 0], desired_line7[desired_level_index][:, 1],
                desired_line8[desired_level_index][:, 1], color='blue', alpha=0.2)

ax.set_title("Southeast Texas Max Still Water Level to River Discharge-Frank")
ax.set_xlabel("Still Water Level (m)")
ax.set_ylabel("Flow (m^3/s)")

ax.set_xlim(0.6, 3.4)
ax.set_ylim(-10, 420)

plt.tight_layout()
plt.savefig(f"Southeast Texas Max Still Water Level to River Discharge SLR Projection Copula-Frank.png", dpi=1200)
plt.show()
